In [1]:
import sys
import time
import os
import tensorflow as tf
import numpy as np

from collections import namedtuple
from tensorflow.python import debug as tf_debug
from tensorflow.core.example import example_pb2
from importlib import reload

import json, os, re, shutil, sys, time
import collections, itertools


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import glob
import random
import struct
import csv
import batch; reload(batch)
import data; reload(data)
import model; reload(model)

from batch import Example,Batch
from data import Vocab
from model import SummarizationModel
from decode import BeamSearchDecoder
from batcher import Batcher
import util
import training_util as tutil




In [3]:
def setup_params():
    hps_dict = {
        'mode' : 'train',
        'single_pass' : False,
        'log_root' : '/home/ubuntu/W266/final_0/W266_Final/model_3/saved/train',
        'exp_name' : '',
        'hidden_dim' : 200,
        'emb_dim' : 128,
        'batch_size' : 100,
        'max_enc_steps' : 400,
        'max_dec_steps' : 100,
        'beam_size' : 4,
        'min_dec_steps' : 35,
        'vocab_size' : 50000,
        'lr' : 0.15,
        'adagrad_init_acc' : 0.1,
        'rand_unif_init_mag' : 0.02,
        'trunc_norm_init_std' : 1e-4,
        'max_grad_norm' : 2.0,
        'pointer_gen' : False,
        'coverage' : False,
        'cov_loss_wt' : 1.0,
        'convert_to_coverage_model' : False,
        'vocab_path' : '/home/ubuntu/W266/final_0/W266_Final/data/final_processed/vocab',
        'data_path' : '/home/ubuntu/W266/final_0/W266_Final/data/final_chunked/train_*'
    }
    
    hps = namedtuple("HParams", hps_dict.keys())(**hps_dict)
    return hps

In [4]:
hps = setup_params()

In [5]:
def train(hps,epochs,train_step,curr_best,best_loss,avg_loss,restore,epoch_start):
    
    lm,vocab,batches,train_dir = tutil.training_init(hps)
    
    with lm.graph.as_default():
        initializer = tf.global_variables_initializer()
        
    with tf.Session(graph=lm.graph) as session:
        
        session.run(initializer)
        saver = tf.train.Saver(max_to_keep=3) # keep 3 checkpoints at a time
        sv = tf.train.Supervisor(logdir=train_dir,
                                 is_chief=True,
                                 saver=saver,
                                 summary_op=None,
                                 save_summaries_secs=30, 
                                 save_model_secs=30, 
                                 global_step=lm.global_step)    

        summary_writer = sv.summary_writer
        
        if restore:
            ckpt_path = util.load_ckpt(saver, session,hps,hps.log_root)

        start_time = time.time()
        for i in range(epoch_start,epoch_start+epochs):
            print(f"[EPOCH {i+1}] Starting training..")
            random.shuffle(batches)
            avg_loss,curr_best,train_step = tutil.run_epoch(lm,
                                                            session,
                                                            batches,
                                                            summary_writer,
                                                            train_dir,
                                                            train_step,
                                                            saver,
                                                            hps,
                                                            best_loss,
                                                            avg_loss)
            
            if(best_loss is None or curr_best < best_loss):
                best_loss = curr_best
            print(f"[EPOCH {i+1}] Complete. Avg Loss: {avg_loss}; Best Loss: {best_loss}")
        sv.stop()
        time_total = tutil.pretty_timedelta(since=start_time)
        print(f"[END] Training complete: Best Loss: {best_loss}; Total time: {time_total}")


In [6]:
avg_loss = 0
best_loss = None
curr_best = best_loss
train_step = 0
epochs = 15
restore = False
epoch_start = 0

train(hps,epochs,train_step,curr_best,best_loss,avg_loss,restore,epoch_start)







max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: 16:03
INFO:tensorflow:Fetching data..
INFO:tensorflow:Creating batches..
INFO:tensorflow:[TOTAL Batches]  : 2808
INFO:tensorflow:[TOTAL Examples] : 280778
INFO:tensorflow:Creating batches..COMPLETE
INFO:tensorflow:Building core graph...
INFO:tensorflow:Adding attention_decoder timestep 0 of 100
INFO:tensorflow:Adding attention_decoder timestep 1 of 100
INFO:tensorflow:Adding attention_decoder timestep 2 of 100
INFO:tensorflow:Adding attention_decoder timestep 3 of 100
INFO:tensorflow:Adding attention_decoder timestep 4 of 100
INFO:tensorflow:Adding attention_decoder timestep 5 of 100
INFO:tensorflow:Adding attention_decoder timestep 6 of 100
INFO:tensorflow:Adding attention_decoder timestep 7 of 100
INFO:tensorflow:Adding attention_decoder timestep 8 of 100
INFO:tensorflow:Adding attention_decoder timestep 9 of 100
INFO:tenso

    [batch 92]: seen 9200 examples : 60.0 eps, Loss: 10.440, Avg loss: 10.700, Best loss: 10.700
    [batch 99]: seen 9900 examples : 60.5 eps, Loss: 10.208, Avg loss: 10.676, Best loss: 10.676
    [batch 106]: seen 10600 examples : 60.8 eps, Loss: 9.626, Avg loss: 10.610, Best loss: 10.610
    [batch 113]: seen 11300 examples : 61.0 eps, Loss: 8.960, Avg loss: 10.507, Best loss: 10.507
    [batch 120]: seen 12000 examples : 61.4 eps, Loss: 8.526, Avg loss: 10.386, Best loss: 10.386
    [batch 127]: seen 12700 examples : 61.7 eps, Loss: 8.517, Avg loss: 10.257, Best loss: 10.257
    [batch 134]: seen 13400 examples : 61.9 eps, Loss: 8.111, Avg loss: 10.121, Best loss: 10.121
    [batch 141]: seen 14100 examples : 62.1 eps, Loss: 7.961, Avg loss: 9.987, Best loss: 9.987
    [batch 148]: seen 14800 examples : 62.4 eps, Loss: 7.914, Avg loss: 9.854, Best loss: 9.854
    [batch 155]: seen 15500 examples : 62.6 eps, Loss: 8.096, Avg loss: 9.726, Best loss: 9.726
    [batch 162]: seen 16200 

    [batch 694]: seen 69400 examples : 66.2 eps, Loss: 7.268, Avg loss: 7.291, Best loss: 7.291
    [batch 701]: seen 70100 examples : 66.2 eps, Loss: 7.319, Avg loss: 7.290, Best loss: 7.289
    [batch 708]: seen 70800 examples : 66.2 eps, Loss: 7.261, Avg loss: 7.289, Best loss: 7.289
    [batch 715]: seen 71500 examples : 66.2 eps, Loss: 7.146, Avg loss: 7.284, Best loss: 7.284
    [batch 722]: seen 72200 examples : 66.3 eps, Loss: 7.284, Avg loss: 7.282, Best loss: 7.282
    [batch 729]: seen 72900 examples : 66.3 eps, Loss: 7.222, Avg loss: 7.280, Best loss: 7.280
    [batch 736]: seen 73600 examples : 66.3 eps, Loss: 7.246, Avg loss: 7.278, Best loss: 7.278
    [batch 743]: seen 74300 examples : 66.3 eps, Loss: 7.319, Avg loss: 7.277, Best loss: 7.277
    [batch 750]: seen 75000 examples : 66.3 eps, Loss: 7.181, Avg loss: 7.276, Best loss: 7.276
    [batch 757]: seen 75700 examples : 66.3 eps, Loss: 7.310, Avg loss: 7.276, Best loss: 7.275
    [batch 764]: seen 76400 examples : 6

    [batch 1289]: seen 128900 examples : 66.9 eps, Loss: 7.220, Avg loss: 7.215, Best loss: 7.213
    [batch 1296]: seen 129600 examples : 67.0 eps, Loss: 7.261, Avg loss: 7.215, Best loss: 7.213
    [batch 1303]: seen 130300 examples : 67.0 eps, Loss: 7.219, Avg loss: 7.215, Best loss: 7.213
    [batch 1310]: seen 131000 examples : 67.0 eps, Loss: 7.275, Avg loss: 7.216, Best loss: 7.213
    [batch 1317]: seen 131700 examples : 67.0 eps, Loss: 7.156, Avg loss: 7.216, Best loss: 7.213
    [batch 1324]: seen 132400 examples : 67.0 eps, Loss: 7.181, Avg loss: 7.216, Best loss: 7.213
    [batch 1331]: seen 133100 examples : 67.0 eps, Loss: 7.230, Avg loss: 7.215, Best loss: 7.213
    [batch 1338]: seen 133800 examples : 67.0 eps, Loss: 7.181, Avg loss: 7.213, Best loss: 7.213
    [batch 1345]: seen 134500 examples : 67.0 eps, Loss: 7.243, Avg loss: 7.212, Best loss: 7.211
    [batch 1352]: seen 135200 examples : 67.0 eps, Loss: 7.185, Avg loss: 7.213, Best loss: 7.211
    [batch 1359]: se

    [batch 1855]: seen 185500 examples : 67.2 eps, Loss: 7.204, Avg loss: 7.200, Best loss: 7.200
    [batch 1862]: seen 186200 examples : 67.2 eps, Loss: 7.282, Avg loss: 7.202, Best loss: 7.200
    [batch 1869]: seen 186900 examples : 67.2 eps, Loss: 7.186, Avg loss: 7.202, Best loss: 7.200
    [batch 1876]: seen 187600 examples : 67.2 eps, Loss: 7.165, Avg loss: 7.203, Best loss: 7.200
    [batch 1883]: seen 188300 examples : 67.2 eps, Loss: 7.167, Avg loss: 7.201, Best loss: 7.200
    [batch 1890]: seen 189000 examples : 67.2 eps, Loss: 7.195, Avg loss: 7.201, Best loss: 7.200
    [batch 1897]: seen 189700 examples : 67.2 eps, Loss: 7.182, Avg loss: 7.201, Best loss: 7.200
    [batch 1904]: seen 190400 examples : 67.2 eps, Loss: 7.189, Avg loss: 7.199, Best loss: 7.199
    [batch 1911]: seen 191100 examples : 67.2 eps, Loss: 7.224, Avg loss: 7.199, Best loss: 7.199
    [batch 1918]: seen 191800 examples : 67.2 eps, Loss: 7.253, Avg loss: 7.199, Best loss: 7.198
    [batch 1925]: se

...

    [batch 1538]: seen 153800 examples : 66.2 eps, Loss: 5.054, Avg loss: 5.111, Best loss: 5.106
    [batch 1545]: seen 154500 examples : 66.3 eps, Loss: 5.234, Avg loss: 5.110, Best loss: 5.106
    [batch 1552]: seen 155200 examples : 66.3 eps, Loss: 5.118, Avg loss: 5.109, Best loss: 5.106
    [batch 1559]: seen 155900 examples : 66.3 eps, Loss: 5.067, Avg loss: 5.108, Best loss: 5.106
    [batch 1566]: seen 156600 examples : 66.3 eps, Loss: 5.112, Avg loss: 5.108, Best loss: 5.106
    [batch 1573]: seen 157300 examples : 66.3 eps, Loss: 5.076, Avg loss: 5.108, Best loss: 5.106
    [batch 1580]: seen 158000 examples : 66.3 eps, Loss: 5.133, Avg loss: 5.110, Best loss: 5.106
    [batch 1587]: seen 158700 examples : 66.3 eps, Loss: 5.248, Avg loss: 5.113, Best loss: 5.106
    [batch 1594]: seen 159400 examples : 66.3 eps, Loss: 5.068, Avg loss: 5.112, Best loss: 5.106
    [batch 1601]: seen 160100 examples : 66.3 eps, Loss: 5.092, Avg loss: 5.112, Best loss: 5.106
    [batch 1608]: se

    [batch 2126]: seen 212600 examples : 66.7 eps, Loss: 5.165, Avg loss: 5.099, Best loss: 5.093
    [batch 2133]: seen 213300 examples : 66.7 eps, Loss: 5.039, Avg loss: 5.100, Best loss: 5.093
    [batch 2140]: seen 214000 examples : 66.7 eps, Loss: 5.006, Avg loss: 5.098, Best loss: 5.093
    [batch 2147]: seen 214700 examples : 66.7 eps, Loss: 5.105, Avg loss: 5.098, Best loss: 5.093
    [batch 2154]: seen 215400 examples : 66.7 eps, Loss: 5.092, Avg loss: 5.097, Best loss: 5.093
    [batch 2161]: seen 216100 examples : 66.7 eps, Loss: 5.115, Avg loss: 5.098, Best loss: 5.093
    [batch 2168]: seen 216800 examples : 66.8 eps, Loss: 5.150, Avg loss: 5.100, Best loss: 5.093
    [batch 2175]: seen 217500 examples : 66.8 eps, Loss: 5.071, Avg loss: 5.099, Best loss: 5.093
    [batch 2182]: seen 218200 examples : 66.8 eps, Loss: 5.054, Avg loss: 5.098, Best loss: 5.093
    [batch 2189]: seen 218900 examples : 66.8 eps, Loss: 5.039, Avg loss: 5.099, Best loss: 5.093
    [batch 2196]: se

    [batch 2714]: seen 271400 examples : 67.0 eps, Loss: 5.114, Avg loss: 5.085, Best loss: 5.084
    [batch 2721]: seen 272100 examples : 67.0 eps, Loss: 5.050, Avg loss: 5.086, Best loss: 5.084
    [batch 2728]: seen 272800 examples : 67.0 eps, Loss: 5.188, Avg loss: 5.088, Best loss: 5.084
    [batch 2735]: seen 273500 examples : 67.0 eps, Loss: 5.147, Avg loss: 5.088, Best loss: 5.084
    [batch 2742]: seen 274200 examples : 67.0 eps, Loss: 5.039, Avg loss: 5.087, Best loss: 5.084
    [batch 2749]: seen 274900 examples : 67.0 eps, Loss: 5.049, Avg loss: 5.085, Best loss: 5.084
    [batch 2756]: seen 275600 examples : 67.0 eps, Loss: 5.027, Avg loss: 5.084, Best loss: 5.084
    [batch 2763]: seen 276300 examples : 67.0 eps, Loss: 5.102, Avg loss: 5.083, Best loss: 5.082
    [batch 2770]: seen 277000 examples : 67.0 eps, Loss: 5.082, Avg loss: 5.084, Best loss: 5.082
    [batch 2777]: seen 277700 examples : 67.0 eps, Loss: 5.076, Avg loss: 5.083, Best loss: 5.082
    [batch 2784]: se

    [batch 499]: seen 49900 examples : 68.3 eps, Loss: 5.004, Avg loss: 5.062, Best loss: 5.056
    [batch 506]: seen 50600 examples : 68.3 eps, Loss: 5.058, Avg loss: 5.060, Best loss: 5.056
    [batch 513]: seen 51300 examples : 68.3 eps, Loss: 4.980, Avg loss: 5.056, Best loss: 5.056
    [batch 520]: seen 52000 examples : 68.3 eps, Loss: 5.110, Avg loss: 5.057, Best loss: 5.056
    [batch 527]: seen 52700 examples : 68.3 eps, Loss: 5.098, Avg loss: 5.057, Best loss: 5.056
    [batch 534]: seen 53400 examples : 68.3 eps, Loss: 4.984, Avg loss: 5.056, Best loss: 5.056
    [batch 541]: seen 54100 examples : 68.3 eps, Loss: 4.950, Avg loss: 5.054, Best loss: 5.054
    [batch 548]: seen 54800 examples : 68.3 eps, Loss: 5.157, Avg loss: 5.057, Best loss: 5.054
    [batch 555]: seen 55500 examples : 68.3 eps, Loss: 5.049, Avg loss: 5.060, Best loss: 5.054
    [batch 562]: seen 56200 examples : 68.3 eps, Loss: 5.116, Avg loss: 5.061, Best loss: 5.054
    [batch 569]: seen 56900 examples : 6

KeyboardInterrupt: 